In [ ]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2, numpy as np
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
import os
import numpy as np
import pandas as pd 
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import keras

In [ ]:
# Deep learning libraries
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
import seaborn as sns
from tensorflow.keras.applications.imagenet_utils import preprocess_input,decode_predictions

In [ ]:
covid_train = "/kaggle/input/covid-shorten/covid19_shorten/covid19/Data/train/COVID19"
normal_train = "/kaggle/input/covid-shorten/covid19_shorten/covid19/Data/train/NORMAL"
pneumonia_train = "/kaggle/input/covid-shorten/covid19_shorten/covid19/Data/train/PNEUMONIA"
lung_opacity="/kaggle/input/covid-shorten/covid19_shorten/covid19/Data/train/LUNG_OPACITY"

len_covid_train = len(os.listdir(covid_train))
len_normal_train = len(os.listdir(normal_train))
len_pneumonia_train =  len(os.listdir(pneumonia_train))
len_lung_opacity= len(os.listdir(lung_opacity))

print("Number of covid Images:",len_covid_train)
print("Number of normal Images:",len_normal_train)
print("Number of pneumonia Images:",len_pneumonia_train)
print("Number of lung_opacity Images:",len_lung_opacity)
print("Total Number of Images:",len_covid_train+len_normal_train+len_pneumonia_train+len_lung_opacity)

In [ ]:
labels = ['COVID-19','Normal', 'pneumonia','Lung Opacity']
img1 = covid_train+"/COVID19(0).jpg"
img2 = normal_train+"/NORMAL(0).jpg"
img3 = pneumonia_train + "/PNEUMONIA(1054).jpg"
img4 = lung_opacity + "/Lung_Opacity-2010.png"

imgs = [img1, img2, img3,img4]

fig, ax = plt.subplots(2,2, figsize=(15, 15))
ax = ax.ravel()
plt.tight_layout()

for i in range(0,4):

    ax[i].imshow(plt.imread(imgs[ i]), cmap='gray')
    ax[i].set_title(labels[i])

In [ ]:
fpath = "/kaggle/input/covid-shorten/covid19_shorten/covid19/Data/train"
random_seed = 42

categories = os.listdir(fpath)
categories = categories[:4]
print("List of categories = ",categories,"\n\nNo. of categories = ", len(categories)) 

In [ ]:
import os
import cv2
from PIL import Image
import numpy as np

def load_images_and_labels(categories, batch_size=32):
    img_lst = []
    labels = []
    for index, category in enumerate(categories):
        img_names = os.listdir(fpath+"/"+category)
        num_batches = len(img_names) // batch_size + int(len(img_names) % batch_size != 0)
        for i in range(num_batches):
            batch_names = img_names[i*batch_size:(i+1)*batch_size]
            batch_imgs = [cv2.imread(fpath+"/"+category+"/"+name) for name in batch_names]
            batch_imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in batch_imgs]
            batch_imgs = [np.array(Image.fromarray(img).resize((224, 224))) for img in batch_imgs]
            img_lst.extend(batch_imgs)
            labels.extend([index]*len(batch_imgs))
    return img_lst, labels


In [ ]:
images, labels = load_images_and_labels(categories)
print("Number of images: ", len(images))
print("Number of labels: ", len(labels))

In [ ]:
images = np.array(images)
labels = np.array(labels)

print("Images shape = ",images.shape,"\nLabels shape = ",labels.shape)
print(type(images),type(labels))

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
images, labels = shuffle(images, labels )
images.resize((images.shape[0],224,224,3))
#images = images.reshape((images.shape[0],1))

from keras.utils import to_categorical
label_output_matrix = to_categorical(labels, num_classes = 4, dtype ="int32")

x_train, x_test, y_train, y_test = train_test_split(images, label_output_matrix, test_size=0.2, random_state=24)


print("x_train shape = ",x_train.shape)
print("y_train shape = ",y_train.shape)
print("\nx_test shape = ",x_test.shape)
print("y_test shape = ",y_test.shape)

In [ ]:
denseNet201_model = tf.keras.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
)

In [ ]:
denseNet201_model.summary()

In [ ]:
for layers in denseNet201_model.layers:
    layers.trainable=True

In [ ]:
print("Number of layers ", len(denseNet201_model.layers))

In [ ]:
for layers in denseNet201_model.layers[:656]:
  layers.trainable = False

In [ ]:
model = tf.keras.models.Sequential()
model.add(denseNet201_model)
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(4,activation = 'softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
import time

start_time = time.time()

history = model.fit(x_train, y_train, validation_data=(x_test, y_test),
                    batch_size=32, epochs=10)

end_time = time.time()
execution_time = end_time - start_time
print("Total execution time for all epochs: {:.2f} seconds".format(execution_time))

In [ ]:
mypredict = model.predict(x_test)
pred = np.argmax(mypredict, axis=-1)
print(pred)
evalute = model.evaluate(x_test, y_test)
print("Accuracy: {:.2f}%".format(evalute[1] * 100))  
print("Loss: {}".format(evalute[0]))

In [ ]:
import cv2
import numpy as np

# Load and preprocess the image
image_path = "/kaggle/input/test-image/WhatsApp Image 2023-08-11 at 12.27.02.jpg"  # Replace with the actual path
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
img = cv2.resize(img, (224, 224))  # Resize to the same dimensions as training images
img = np.expand_dims(img, axis=0)  # Add a batch dimension
img = img / 255.0  # Normalize pixel values

# Make predictions
predictions = model.predict(img)
predicted_class_index = np.argmax(predictions, axis=-1)[0]

# Map index to label
labels = ['COVID-19', 'Normal', 'Pneumonia', 'Lung Opacity']
predicted_label = labels[predicted_class_index]

print("Predicted Label:", predicted_label)

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Make predictions on the test set
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Compute confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Create a heatmap of the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=categories, yticklabels=categories)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
x_output=model.predict(x_train)

In [ ]:
#Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_output, y_train)
y_pred_rf = rf.predict(mypredict)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf)*100)
print("Random Forest Precision:", precision_score(y_test, y_pred_rf,average='micro')*100)
print("Random Forest Recall:", recall_score(y_test, y_pred_rf,average='micro')*100)
print("Random Forest F1-score:", f1_score(y_test, y_pred_rf,average='micro')*100)

rf_acc=accuracy_score(y_test, y_pred_rf)
rf_pre=precision_score(y_test, y_pred_rf,average='micro')
rf_rec=recall_score(y_test, y_pred_rf,average='micro')
rf_f1=f1_score(y_test, y_pred_rf,average='micro')

In [ ]:
#Decision Tree Classifier
dt = DecisionTreeClassifier()
dt.fit(x_output, y_train)
y_pred_dt = dt.predict(mypredict)

print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt)*100)
print("Decision Tree Precision:", precision_score(y_test, y_pred_dt,average='micro')*100)
print("Decision Tree Recall:", recall_score(y_test, y_pred_dt,average='micro')*100)
print("Decision Tree F1-score:", f1_score(y_test, y_pred_dt,average='micro')*100)

dt_acc=accuracy_score(y_test, y_pred_dt)
dt_pre=precision_score(y_test, y_pred_dt,average='micro')
dt_rec=recall_score(y_test, y_pred_dt,average='micro')
dt_f1=f1_score(y_test, y_pred_dt,average='micro')

In [ ]:
# K-Nearest Neighbors Classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_output, y_train)
y_pred_knn = knn.predict(mypredict)

print("K-Nearest Neighbors Accuracy:", accuracy_score(y_test, y_pred_knn)*100)
print("K-Nearest Neighbors Precision:", precision_score(y_test, y_pred_knn, average='micro')*100)
print("K-Nearest Neighbors Recall:", recall_score(y_test, y_pred_knn, average='micro')*100)
print("K-Nearest Neighbors F1-score:", f1_score(y_test, y_pred_knn, average='micro')*100)

knn_acc = accuracy_score(y_test, y_pred_knn)
knn_pre = precision_score(y_test, y_pred_knn, average='micro')
knn_rec = recall_score(y_test, y_pred_knn, average='micro')
knn_f1 = f1_score(y_test, y_pred_knn, average='micro')


In [ ]:
# Extra Trees Classifier
etc = ExtraTreesClassifier()
etc.fit(x_output, y_train)
y_pred_etc = etc.predict(mypredict)

print("Extra Trees Classifier Accuracy:", accuracy_score(y_test, y_pred_etc)*100)
print("Extra Trees Classifier Precision:", precision_score(y_test, y_pred_etc, average='micro')*100)
print("Extra Trees Classifier Recall:", recall_score(y_test, y_pred_etc, average='micro')*100)
print("Extra Trees Classifier F1-score:", f1_score(y_test, y_pred_etc, average='micro')*100)

etc_acc = accuracy_score(y_test, y_pred_etc)
etc_pre = precision_score(y_test, y_pred_etc, average='micro')
etc_rec = recall_score(y_test, y_pred_etc, average='micro')
etc_f1 = f1_score(y_test, y_pred_etc, average='micro')


In [ ]:
from prettytable import PrettyTable

table = PrettyTable()
table.field_names = ["Model", "Accuracy", "Precision", "Recall", "F1-score"]

table.add_row(["Random Forest", rf_acc*100, rf_pre*100, rf_rec*100, rf_f1*100])
table.add_row(["Decision Tree", dt_acc*100, dt_pre*100, dt_rec*100, dt_f1*100])
table.add_row(["KNN", knn_acc*100, knn_pre*100, knn_rec*100, knn_f1*100])
table.add_row(["Extra Trees", etc_acc*100, etc_pre*100, etc_rec*100, etc_f1*100])

print(table)
